In [1]:
import pandas as pd

# 10-K Reports

In [2]:
tenkreports = pd.read_parquet('datasets/Predictors/mda_text.parquet')

In [3]:
tenkreports

,submission_type,filing_date,report_date,report_year,cik,company_conformed_name,text
0,10-K,20160311,20151231,2015,0000002178,"ADAMS RESOURCES & ENERGY, INC.",Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS O...
1,10-K,20160218,20151226,2015,0000002488,ADVANCED MICRO DEVICES INC,ITEM 7.\n\nMANAGEMENT'S DISCUSSION AND ANALYSI...
2,10-K,20160216,20151231,2015,0000003499,ALEXANDERS INC,ITEM 7. management's discussion and analys...
3,10-K,20151210,20150930,2015,0000003545,ALICO INC,Item 7. Management's Discussion and Analysis o...
4,10-K,20161206,20160930,2016,0000003545,ALICO INC,Item 7. Management's Discussion and Analysis o...
...,...,...,...,...,...,...,...
8317,10-K,20161108,20160731,2016,0001550053,"Grand Perfecta, Inc.",ITEM 7. MANAGEMENT'S DISCUSSION AND ANAL...
8318,10-K,20161114,20160930,2016,0001023459,SIMULATIONS PLUS INC,ITEM 7 – MANAGEMENT'S\nDISCUSSION AND ANALYSIS...
8319,10-K,20161213,20160831,2016,0001343009,Cannabics Pharmaceuticals Inc.,Item 7. Management's\nDiscussion and Analysis ...
8320,10-K,20161214,20160831,2016,0001313938,"Sibannac, Inc.",ITEM 7. MANAGEMENT'S DISCUSSION AND\nANALYSIS ...


In [4]:
tenkreports['report_year'].value_counts()

report_year
2015    6467
2016    1684
2014     131
2013      24
2012      10
2011       2
2010       2
2008       1
2009       1
Name: count, dtype: int64

We only have data for 2015.

# Chen-Zimmerman data

chenz = pd.read_csv('datasets/Predictors/chenz.csv')
chenz
# Earning Calls

In [6]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')

In [10]:
earnings_calls

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section
...,...,...,...,...,...,...,...
12475891,1220890,45,So a key driver of the improvement last year a...,2017-05-11,220426,Zurich Insurance Group AG,Answer
12475892,1220890,43,"On the rate increase, I mean if you look at th...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475893,1220890,40,"So I'm going to apologize in advance, Ralph. I...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475894,1220890,37,"Yes, thanks. So there -- I mean, given what I ...",2017-05-11,220426,Zurich Insurance Group AG,Answer


In [9]:
earnings_calls.dtypes

transcriptid                            int64
componentorder                          int64
componenttext                          object
mostimportantdateutc           datetime64[us]
gvkey                                  object
companyname                            object
transcriptcomponenttypename            object
dtype: object

# Quarterly Compustat Firm Characteristics

In [11]:
firm_char = pd.read_csv('datasets/Predictors/CompFirmCharac.csv')

/var/folders/ft/f5drds915nl_7p2g4tg67z840000gn/T/ipykernel_51294/2667926953.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  firm_char = pd.read_csv('datasets/Predictors/CompFirmCharac.csv')


In [12]:
firm_char

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqpy,xoptepsy,xoptqpy,xopty,xrdy,xsgay,exchg,cik,costat,fic
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,I,USA


# JKP

In [13]:
jkp = pd.read_csv('datasets/Predictors/jkp.csv')

In [14]:
jkp

,location,name,freq,weighting,direction,n_stocks,n_stocks_min,date,ret
0,usa,age,monthly,vw_cap,-1,502,8,1926-03-31,-0.086738
1,usa,age,monthly,vw_cap,-1,505,12,1926-04-30,-0.000572
2,usa,age,monthly,vw_cap,-1,510,17,1926-05-31,-0.006332
3,usa,age,monthly,vw_cap,-1,512,21,1926-06-30,-0.003012
4,usa,age,monthly,vw_cap,-1,503,23,1926-07-31,0.003211
...,...,...,...,...,...,...,...,...,...
144615,usa,zero_trades_252d,monthly,vw_cap,1,3627,1622,2024-08-31,0.031409
144616,usa,zero_trades_252d,monthly,vw_cap,1,3631,1630,2024-09-30,-0.011177
144617,usa,zero_trades_252d,monthly,vw_cap,1,3614,1634,2024-10-31,-0.008210
144618,usa,zero_trades_252d,monthly,vw_cap,1,3595,1650,2024-11-30,-0.052747


idée: C'est que le sentiment analysis va retourner uniquement un sentiment: positif, négatif ou neutre donc il n'y aura pas vraiment de data leakage. Dans les faits, dans certains datasets ils parlent des performances mais on est pas dans le data leakage car notre sentiment analysis ne resort pas de chiffre non?

# Earning Calls

In [2]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')
earnings_calls.head()

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section


In [3]:
earnings_calls

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section
...,...,...,...,...,...,...,...
12475891,1220890,45,So a key driver of the improvement last year a...,2017-05-11,220426,Zurich Insurance Group AG,Answer
12475892,1220890,43,"On the rate increase, I mean if you look at th...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475893,1220890,40,"So I'm going to apologize in advance, Ralph. I...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475894,1220890,37,"Yes, thanks. So there -- I mean, given what I ...",2017-05-11,220426,Zurich Insurance Group AG,Answer


In [4]:
earnings_calls.dtypes

transcriptid                            int64
componentorder                          int64
componenttext                          object
mostimportantdateutc           datetime64[us]
gvkey                                  object
companyname                            object
transcriptcomponenttypename            object
dtype: object

In [ ]:
earnings_calls_sorted = earnings_calls.sort_values(by=['mostimportantdateutc','transcriptid', 'componentorder'])

In [14]:
earnings_calls

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
11515141,15674,69,We haven’t changed. Our long term rate of retu...,2010-01-27,125533,Pactiv LLC,Answer
8833246,15674,87,"Okay. Ed, I wanted to just come back just so I...",2010-01-27,125533,Pactiv LLC,Question
11515181,15674,44,In the industry volume?\n,2010-01-27,125533,Pactiv LLC,Answer
8833248,15674,85,"Okay, great. Thanks, guys.\n",2010-01-27,125533,Pactiv LLC,Question
8833252,15674,83,Okay. And then just one quick follow up. Withi...,2010-01-27,125533,Pactiv LLC,Question
...,...,...,...,...,...,...,...
6333756,3346142,72,"And do you think -- speaking of that Andy, do ...",2019-10-16,004723,U.S. Bancorp,Question
6333782,3346142,70,You guys have been very good at sharing with u...,2019-10-16,004723,U.S. Bancorp,Question
6333814,3346142,67,So you guys addressed a lot of my questions. I...,2019-10-16,004723,U.S. Bancorp,Question
6333874,3346142,61,Just stepping back kind of bigger picture in t...,2019-10-16,004723,U.S. Bancorp,Question


In [ ]:
earnings_calls_sorted.to_parquet('datasets/Predictors/earnings_calls_sorted')